In [ ]:
### %matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme="onedork", context="notebook", ticks=True, grid=True)

import numpy as np
import h5py
import os
import sys
import glob

import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
import pandas as pd
import tikzplotlib

from tqdm import tqdm
import helper.circular

sim_path = "output/simulation/*.h5"
out_file = "output/analysis/"

In [ ]:
if False or not os.path.isfile(os.path.join(out_file, "pm.pkl")):
    os.makedirs("output/analysis", exist_ok=True)
    file_list = sorted(glob.glob(sim_path))

    # get resolution:
    with h5py.File(file_list[0], 'r') as h5f:
        for name in ['PM']:
            h5f_sub = h5f[name + '/p']
            res = h5f_sub['analysis/rofl/direction'].shape
    shape = (len(file_list), 2, res[0] * res[1])

    df = {'PM': pd.DataFrame(), 'LAP': pd.DataFrame()}

    for f, file in enumerate(tqdm(file_list)):
        with h5py.File(file, 'r') as h5f:
            for name in ['PM', 'LAP']:
                for m, model in enumerate(['p', 'r']):
                    h5f_sub = h5f[name + '/' + model]

                    epa_dir = h5f_sub['analysis/epa/0/direction'][...].ravel()
                    # epa_dir = helper.circular.remap(epa_dir, 0, np.pi)
                    epa_dir = helper.circular.center_mean(epa_dir, 0, np.pi)

                    rofl_dir = h5f_sub['analysis/rofl/direction'][...].ravel()
                    rofl_dir[rofl_dir > np.pi / 2] -= np.pi

                    df[name] = df[name].append(
                        {
                            "f0":
                            h5f_sub.attrs['parameter/f0_inc'],
                            "f1":
                            h5f_sub.attrs['parameter/f1_rot'],
                            "omega":
                            h5f_sub.attrs['parameter/omega'],
                            "psi":
                            h5f_sub.attrs['parameter/psi'],
                            "rofl_dir":
                            np.rad2deg(rofl_dir),
                            "rofl_inc":
                            h5f_sub['analysis/rofl/inclination'][...].ravel(),
                            "rofl_trel":
                            h5f_sub['analysis/rofl/t_rel'][...].ravel(),
                            "epa_trans":
                            h5f_sub['analysis/epa/0/transmittance'][
                                ...].ravel(),
                            "epa_dir":
                            np.rad2deg(epa_dir),
                            "epa_ret":
                            h5f_sub['analysis/epa/0/retardation'][...].ravel(),
                        },
                        ignore_index=True)

    df['PM'].to_pickle(os.path.join(out_file, "pm.pkl"))
    df['LAP'].to_pickle(os.path.join(out_file, "lap.pkl"))

df_pm = pd.read_pickle(os.path.join(out_file, "pm.pkl"))
df_lap = pd.read_pickle(os.path.join(out_file, "lap.pkl"))

In [ ]:
data=df_pm[(df_pm.f0 == 0.0) & (df_pm.f1 == 0.0)].explode("epa_dir")
grid = sns.FacetGrid(data, col="omega", hue="omega", palette=sns.color_palette("dark", 10),
                     col_wrap=5, height=2.5)
grid.map(plt.scatter, "psi", "epa_dir", marker="o")
grid.fig.tight_layout(w_pad=1)

In [ ]:
data=df_pm[(df_pm.f0 == 0.0) & (df_pm.f1 == 0.0)].explode("epa_dir")

fig, axes = plt.subplots(2, 5)
for i, omega in enumerate(data.omega.unique()):
    sns.boxplot(x="psi",
                y="epa_dir",
                data=data[data.omega == omega],
                orient='v',
                color = sns.color_palette("dark", 10)[i],
                ax=axes[i // 5, i % 5])
    
data=df_pm[(df_pm.f0 == 0.0) & (df_pm.f1 == 0.0)].explode("epa_ret")
fig, axes = plt.subplots(2, 5)
for i, omega in enumerate(data.omega.unique()):
    sns.boxplot(x="psi",
                y="epa_ret",
                data=data[data.omega == omega],
                orient='v',
                color = sns.color_palette("dark", 10)[i],
                ax=axes[i // 5, i % 5])

In [ ]:
sns.stripplot(x="psi",
                y="epa_dir",
                data=data[data.omega == omega],
                orient='v', jitter=0.5)